# Use examples of [premise](https://github.com/romainsacchi/premise)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to create life cycle invneotries databases for prospective environmental impact assessment.

This library extract useful information from IAM model output files (such as those of REMIND or IMAGE) and aligns inventories in the ecoinvent database accordingly.

With version 0.2.0, the following transformation are available:

* `update_electricity_to_iam_data()`: create regional electricity markets and adjust efficiency of power plants
* `update_cement_to_iam_data()`: creates regional markets for clinker production and adjust clinker production efficiency
* `update_steel_to_iam_data()`: creates regional markets for steel and adjust steel production efficiency
* `update_cars()`: updates car inventories and fleet average car transport activities
* `update_trucks()`: updates truck inventories and fleet average trucks transport activities
* `update_solar_PV()`: updates efficeincy of solar PV modules

Additional documentation on the moethodology is available [here](https://premise.readthedocs.io/en/latest/introduction.html).

## Requirements

* Pyhton 3.7 or above
* a user license for ecoinvent v.3

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have a open `brightway2` project with abiosphere as well as an
ecoinvent v.3 database registered in that project.

In [ ]:
from premise import *
import brightway2 as bw

In [ ]:
bw.projects.set_current("import_36_for_carculator")

In [ ]:
del bw.databases["ecoinvent_image_SSP2-Base_2028"]
del bw.databases["ecoinvent_remind_SSP2-Base_2032"]

In [ ]:
bw.databases

### List of available scenarios

Some scenarios come installed with the library.
They are stored in `data/iam_ouput_files` from the root directory.
They are all within the SSP2 pathway (middle of the road).

With REMIND:
* "SSP2-Base"
* "SSP2-NPi"
* "SSP2-NDC"
* "SSP2-PkBudg1300"
* "SSP2-PkBudg1100"
* "SSP2-PkBudg900"

With IMAGE:
* "SSP2-Base"


### Database creation from default scenarios

To create a scenario using REMIND's SSP2 Base pathway, from ecoinvent 3.6 for the year 2028, one would execute the following cell. This leads to the extraction of the database, some cleanup as well as importing a few additional inventories.

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2028}
            ],
            source_db="ecoinvent 3.6 cutoff", # <-- name of the database in the BW2 project. Must be a string.
            source_version="3.6", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.7.1". Must be a string.
    )

or here with ecoinvent 3.7.1

In [ ]:
ndb = NewDatabase(
        scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2028}
            ],
        source_db="ecoinvent 3.7 cutoff", # <-- this is NEW.
        source_version="3.7.1", # <-- this is NEW
    )

If you want to create multiple databases at once, just populate the `scenarios` list.

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2028},
                {"model":"remind", "pathway":"SSP2-NPi", "year":2032}
            ],
            source_db="ecoinvent 3.5 cutoff", # <-- name of the database. Must be a string.
            source_version="3.5", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.7.1"
                 )

When the database is loaded and the additional inventories imported, you can apply a transformation function.
For example here, we adjust the efficiency of the solar PVs to the two scenarios we have loaded.
We go more in details later.

In [ ]:
ndb.update_solar_PV()

And then, we register these two databases back into brightway2.

In [ ]:
ndb.write_db_to_brightway()

### Database creation from non-default scenarios

If you have some specific IAM scenarios (one that is not included in `premise`) you would like to build a database from, you can specify the directory to those.

**Important remark**: your scenario file must begin with "remind_" or "image_".

In [ ]:
ndb = NewDatabase(
    scenarios = [{"model":"remind", "pathway":"my_special_scenario", "year":2028, "filepath":r"C:\Users\sacchi_r\Downloads\REMIND"}],        
    source_db="ecoinvent 3.6 cutoff", # <-- name of the database
    source_version=3.6, # <-- version of ecoinvent
                 )

# Use case with ecospold2

The source database does not have to be from a brightway2 project.
It can be directly extracted from the bunch of ecospold2 files one gets when downloaded from the [ecoinvent website](https://ecoinvent.org).

For this, one needs to specify the argument `source_db = "ecospold"` as well as `source_file_path`, which is the directory leading to the ecospold files.

For example, here we combine the use of a specific (non-default) IAM scenario file with the use of ecospold2 files as data source (ecoinvent 3.5 in this case).

In [ ]:
ndb = NewDatabase(
        scenarios = [
            {"model":"remind", "pathway":"my_special_scenario", "year":2028, "filepath":r"C:\Users\sacchi_r\Downloads\REMIND"}
        ],        
        source_type="ecospold", # <--- this is NEW
        source_file_path=r"C:\Users\sacchi_r\Dropbox\Public\ecoinvent 3.5_cutoff_ecoSpold02\datasets", # <-- this is NEW
        source_version=3.5,
    )

# Transformation functions

These functions modify the extracted database:

* **update_electricity_to_iam_data()**: alignment of regional electricity production mixes as well as efficiencies for a number of
  electricity production technologies, including Carbon Capture and Storage technologies.
* **update_cement_to_iam_data()**: adjustment of technologies for cement production (dry, semi-dry, wet, with pre-heater or not),
  fuel efficiency of kilns, fuel mix of kilns (including biomass and waste fuels) and clinker-to-cement ratio.
* **update_steel_to_iam_data()**: adjustment of process efficiency, fuel mix and share of secondary steel in steel markets.
* **update_solar_PV()**: adjustment of solar PV panels efficiency to the year considered.

They can be applied *separately*, *consecutively* or *altogether* (using instead **.update_all()**).
They will apply to all the scenario-specific databases listed in `scenarios`.

In [ ]:
ndb.update_all()

#### Additional options for database creation with cars and truck inventories

Two additional functions are available:
* **update_cars()**: creates updated inventories for fleet average passenger cars and links back to activities that consume
transport.
* **update_trucks()**: creates updated inventories for fleet average lorry trucks and links back to activities that consume
transport.


By default, inventories for passenger cars and trucks are not added upon the creation of the database.
But you can add them by adding the following keys to the scenario dictionary: `passenger cars` and `trucks`.
This will create inventories for passenger cars and trucks for each region of the IAM model.

For example, here we create car inventories in the first scenario, and both cars and trucks inventories in the
second scenario. Leaving an empty dicitonary indicates that we do not apply any filter in terms of regions and that we do not provide a fleet composition file (a default one is then used).

In [1]:
from premise import *
import brightway2 as bw
bw.projects.set_current("import_36_for_carculator")

In [2]:
bw.databases

Databases dictionary with 6 object(s):
	additional_biosphere
	biosphere3
	carculator db
	ecoinvent 3.5 cutoff
	ecoinvent 3.6 cutoff
	ecoinvent 3.7 cutoff

In [ ]:
del bw.databases["ecoinvent_remind_SSP2-Base_2032"]

In [3]:
ndb = NewDatabase(
            scenarios=[
                {"model":"image", "pathway":"SSP2-Base", "year":2028,},
                #{"model":"remind", "pathway":"SSP2-Base", "year":2032}
            ],
            source_db="ecoinvent 3.7 cutoff", # <-- name of the database. Must be a string.
            source_version="3.7", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.7.1"
                 )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|█████████████████████████████████| 19128/19128 [00:00<00:00, 60037.09it/s]


Adding exchange data to activities


100%|███████████████████████████████| 621717/621717 [00:44<00:00, 13959.28it/s]


Filling out exchange data


100%|██████████████████████████████████| 19128/19128 [00:03<00:00, 5013.36it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Remove empty exchanges.

/////////////////// IMPORTING ADDITIONAL INVENTORIES ////////////////////
Importing necessary inventories...

Done!



In [4]:
ndb.update_solar_PV()


/////////////////// SOLAR PV ////////////////////
Update efficiency of solar PVs.



In [ ]:
ndb.update_all()

In [5]:
from premise import utils

In [6]:
scenarios = utils.add_modified_tags(ndb.db, ndb.scenarios)

[('photovoltaic slanted-roof installation, 3kWp, ribbon-Si, panel, mounted, on roof', 'photovoltaic slanted-roof installation, 3kWp, ribbon-Si, panel, mounted, on roof', 'unit', 'CH'), ('photovoltaic slanted-roof installation, 3kWp, single-Si, panel, mounted, on roof', 'photovoltaic slanted-roof installation, 3kWp, single-Si, panel, mounted, on roof', 'unit', 'CH'), ('photovoltaic slanted-roof installation, 3kWp, a-Si, laminated, integrated, on roof', 'photovoltaic slanted-roof installation, 3kWp, a-Si, laminated, integrated, on roof', 'unit', 'RoW'), ('photovoltaic slanted-roof installation, 3kWp, multi-Si, laminated, integrated, on roof', 'photovoltaic slanted-roof installation, 3kWp, multi-Si, laminated, integrated, on roof', 'unit', 'CH'), ('photovoltaic slanted-roof installation, 3kWp, a-Si, panel, mounted, on roof', 'photovoltaic slanted-roof installation, 3kWp, a-Si, panel, mounted, on roof', 'unit', 'RoW'), ('photovoltaic flat-roof installation, 3kWp, multi-Si, on roof', 'photo

In [7]:
for ds in scenarios[0]["database"]:
    if "modified" in ds:
        print(ds["name"])
        
    for exc in ds["exchanges"]:
        if "modified" in exc:
            print(exc["name"])

In [ ]:
for ds in scenarios[0]["database"]:
    for exc in ds["exchanges"]:
        if exc["type"]=="biosphere":
            print(exc)

In [ ]:
from premise import export
exp = export.Export(ndb.db)

In [ ]:
ind_B = export.create_index_of_B_matrix()
ind_B

In [ ]:
for ds in scenarios[1]["database"]:
    for exc in ds["exchanges"]:
        if "modified" in exc:
            if exc["type"]=="technosphere":
                print(exc["name"])

In [ ]:
for x, y in d[1].items():
    if x not in d[0]:
        print(x)
    if d[1][x] != d[0][x]:
        print(x)

In [ ]:
ndb.update_trucks()

In [ ]:
ndb.scenarios[0]

If you want to limit the inventories of passenger cars and trucks to specific regions, you need to pass a dictionary instead, like so:

In [ ]:
ndb = NewDatabase(
            model="remind", # <-- name of the IAM model
            scenario="SSP2-Base", # <-- name of the scenario
            year=2028, # <-- year
            source_db="ecoinvent 3.6 cutoff", # <-- name of the database
            source_version=3.6, # <-- version of ecoinvent
            add_passenger_cars = {"region":["EUR", "USA"]}, # <-- only for EU and US
            add_trucks = {"region": ["CHA"]} # <-- only for China
                 )

### Fleet files

A last word about passenger cars: it is possible to pass a fleet composition file to generate fleet average inventories and limit those to specific regions (here, the European region).

**Important remark**: fleet avergae invenotries generation from fleet compoistion files works only with REMIND at the moment.

In [ ]:
ndb = NewDatabase(
            model="remind",
            scenario="SSP2-Base", 
            year=2028,
            source_db="ecoinvent 3.6 cutoff",
            source_version=3.6, 
            add_passenger_cars={
                "fleet file": r"C:\Users\sacchi_r\Downloads\REMIND\Budg1100_ConvSyn_vintcomp.csv", # <-- this is NEW
                "region": ["EUR"]
        },
     )

# Export

### As a Brightway2 database

Export the modified database to brightway2

In [ ]:
ndb.write_db_to_brightway()

### As a sparse matrix representation

Or export it as a sparse matrix representation.

This will export four files:

* "A_matrix.csv": matrix coordinates and values of shape (index of activity; index of product; value) for the technosphere
* "A_matrix_index.csv": labels for indices for A matrix of shape (name of activity, reference product, unit, location, index)
* "B_matrix.csv": matrix coordinates and values of shape (index of activity; index of biosphere flow; value) for the biosphere
* "B_matrix_index.csv": labels for indices for B matrix of shape (name of biosphere flow, main compartment, sub-compartmnet, unit, index)

As a convenience, you can specifiy a directory where to store the exported matrices.
If the directory does not exist, it will be created.
If you leave it unspecified, they will be stored in **data/matrices** in the root folder of the library.

In [ ]:
ndb.write_db_to_matrices(filepath=r"C:/Users/sacchi_r/Downloads/exported_matrices")

### As a SimaPro CSV file

In [ ]:
ndb.write_db_to_simapro(filepath=r"C:/Users/sacchi_r/Downloads/exported_simapro_file")